<a href="https://colab.research.google.com/github/ssegovba/identifying-deforestation/blob/main/cv_final_project_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building on top of a pre-trained model

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image

from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/Shareddrives/computer-vision-project/Data/Unzipped/planet/planet/"
train_path = "/content/drive/Shareddrives/computer-vision-project/Data/Unzipped/planet/planet/train-jpg/"
test_path = "/content/drive/Shareddrives/computer-vision-project/Data/Unzipped/planet/planet/test-jpg"

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os

## santi's code here:

In [ ]:
# Load the CSV file with the metadata
labels_df = pd.read_csv(data_path + 'train_classes.csv')

# Add extension so image names match file names
labels_df['image_name'] = labels_df['image_name'].apply(lambda x: x + '.jpg')

# Convert the space-separated tags into a list of tags
labels_df['tags'] = labels_df['tags'].apply(lambda x: x.split())

# Optional: Use a MultiLabelBinarizer for the tags if needed later for model training
mlb = MultiLabelBinarizer()
labels_df['encoded_tags'] = list(mlb.fit_transform(labels_df['tags']))

In [ ]:
train_df = labels_df
train_df["image_name"] = train_path + labels_df["image_name"]
train_df

,image_name,tags,encoded_tags
0,/content/drive/Shareddrives/computer-vision-pr...,"[haze, primary]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
1,/content/drive/Shareddrives/computer-vision-pr...,"[agriculture, clear, primary, water]","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,/content/drive/Shareddrives/computer-vision-pr...,"[clear, primary]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,/content/drive/Shareddrives/computer-vision-pr...,"[clear, primary]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,/content/drive/Shareddrives/computer-vision-pr...,"[agriculture, clear, habitation, primary, road]","[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, ..."
...,...,...,...
40474,/content/drive/Shareddrives/computer-vision-pr...,"[clear, primary]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
40475,/content/drive/Shareddrives/computer-vision-pr...,[cloudy],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
40476,/content/drive/Shareddrives/computer-vision-pr...,"[agriculture, clear, primary]","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
40477,/content/drive/Shareddrives/computer-vision-pr...,"[agriculture, clear, primary, road]","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."


In [ ]:
train_df["image_name"][0]

'/content/drive/Shareddrives/computer-vision-project/Data/Unzipped/planet/planet/train-jpg/train_0.jpg'

In [ ]:
# Initialize the ImageDataGenerator with any specific augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
image_height = 150
image_width = 150
batch_size = 64

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_name',
    y_col='tags',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)


Found 37620 validated image filenames belonging to 17 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 2859 invalid image filename(s) in x_col="image_name". These filename(s) will be ignored.
  warnings.warn(


## santi's part of the code

In [ ]:
def fetch_images_and_labels(dataset, directory, batch_size=32):  # New parameter: batch_size
    n = len(dataset)
    while True:
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch_images = []
            batch_labels = []
            for _, row in dataset.iloc[start:end].iterrows():
                file_path = os.path.join(directory, row['image_name'])
                img = image.load_img(file_path, target_size=(128, 128))
                img_array = image.img_to_array(img)
                batch_images.append(img_array)
                batch_labels.append(row['encoded_tags'])

            # Convert lists to Numpy arrays
            batch_images = np.array(batch_images)
            batch_labels = np.array(batch_labels)

            # Apply transformations
            yield train_datagen.flow(batch_images, batch_labels, batch_size=batch_size).next()

# Create a generator for training data
train_generator = fetch_images_and_labels(train_df, train_path, batch_size=32)

In [ ]:
train_generator

<generator object fetch_images_and_labels at 0x7f6f0629be60>

## back to my part

In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
# load the VGG model without the top layers (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# add top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # units according to data
predictions = Dense(17, activation='sigmoid')(x)  # 17 is the number of classes

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
model.fit(train_generator, epochs=5)


Epoch 1/5
 15/588 [..............................] - ETA: 3:26:18 - loss: 0.5279 - accuracy: 0.0823

KeyboardInterrupt: 